# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [147]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import gkey


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [149]:
df = pd.read_csv(r'../WeatherPy/Output/cities.csv')
df['Hotel Name'] = " "
df.head()

,Unnamed: 0,City,Country,Date,Latitude,Longitude,Max Temp,Cloudiness,Humidity,Wind Speed,Hotel Name
0,0,Atuona,PF,1569413196,-9.80,-139.03,79.35,16,78,22.37,
1,1,Castro,CL,1569413196,-42.48,-73.76,48.20,90,100,6.93,
2,2,Jamestown,AU,1569413197,-33.21,138.60,49.47,82,81,3.15,
3,3,New Norfolk,AU,1569413198,-42.78,147.06,51.80,40,62,11.41,
4,4,Vaini,IN,1569413198,15.34,74.49,82.40,40,74,9.17,


In [150]:
del df['Unnamed: 0']
df.head()

,City,Country,Date,Latitude,Longitude,Max Temp,Cloudiness,Humidity,Wind Speed,Hotel Name
0,Atuona,PF,1569413196,-9.80,-139.03,79.35,16,78,22.37,
1,Castro,CL,1569413196,-42.48,-73.76,48.20,90,100,6.93,
2,Jamestown,AU,1569413197,-33.21,138.60,49.47,82,81,3.15,
3,New Norfolk,AU,1569413198,-42.78,147.06,51.80,40,62,11.41,
4,Vaini,IN,1569413198,15.34,74.49,82.40,40,74,9.17,


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [151]:
locations = df[['Latitude','Longitude']]
weights = df['Humidity']
fig = gmaps.figure()
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights))
fig.max_intensity = df['Humidity'].max()
fig.point_radius = 5
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [157]:
hotel_df = df.loc[(df['Max Temp']>75) & (df['Humidity']<50) & (df['Wind Speed']<7)]
hotel_df.reset_index(inplace=True)
del hotel_df['index']
hotel_df

,City,Country,Date,Latitude,Longitude,Max Temp,Cloudiness,Humidity,Wind Speed,Hotel Name
0,Gao,ML,1569413201,16.28,-0.04,102.94,0,14,6.82,
1,Saint George,GR,1569413262,39.45,22.34,78.80,20,47,6.93,
2,Kununurra,AU,1569413265,-15.77,128.74,75.20,0,38,5.48,
3,Hermanus,ZA,1569413327,-34.42,19.24,84.00,0,32,3.00,
4,Taoudenni,ML,1569413333,22.68,-3.98,107.07,0,7,2.04,
5,Bilma,NE,1569413340,18.69,12.92,99.70,0,10,6.64,
6,Marrakesh,MA,1569413341,31.63,-7.99,89.60,20,29,5.82,
7,Zaria,NG,1569413352,11.11,7.72,89.07,97,47,3.91,
8,Bunia,CD,1569413401,1.56,30.25,86.20,46,45,1.77,
9,Malanje,AO,1569413463,-9.54,16.35,83.32,96,47,5.66,


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [159]:
url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
for index, row in ideal_locs.iterrows():
    name = hotel_df.loc[index,'City'] 
    lat = hotel_df.loc[index,'Latitude']
    lng = hotel_df.loc[index,'Longitude']
    parameters = {
        'key':gkey,
        'keyword':'hotel',
        'language':'en',
        'rankby':'prominence',
        'radius': 5000,
        'location':(f'{lat},{lng}')
    } 
    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {name}.")
    response = requests.get(url, params=parameters).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"The closest Hotel to {name} is {results[0]['name']}.")
        
        hotel_df.loc[index,'Hotel Name'] = results[0]['name']
              
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        hotel_df.loc[index,'Hotel Name'] = np.nan
        
    print("------------")

Retrieving Results for Index 0: Gao.
The closest Hotel to Gao is Hôtel Atlantide.
------------
Retrieving Results for Index 1: Saint George.
Missing field/result... skipping.
------------
Retrieving Results for Index 2: Kununurra.
The closest Hotel to Kununurra is Hotel Kununurra.
------------
Retrieving Results for Index 3: Hermanus.
The closest Hotel to Hermanus is Windsor Hotel.
------------
Retrieving Results for Index 4: Taoudenni.
The closest Hotel to Taoudenni is Bir Bekhera.
------------
Retrieving Results for Index 5: Bilma.
The closest Hotel to Bilma is Bilma.
------------
Retrieving Results for Index 6: Marrakesh.
The closest Hotel to Marrakesh is Riad Adore by Pure Riads.
------------
Retrieving Results for Index 7: Zaria.
The closest Hotel to Zaria is Luziana Hotel.
------------
Retrieving Results for Index 8: Bunia.
The closest Hotel to Bunia is Hotel Karibuni.
------------
Retrieving Results for Index 9: Malanje.
The closest Hotel to Malanje is Hotel Portugalia.
--------

KeyError: 18

In [160]:
hotel_df

,City,Country,Date,Latitude,Longitude,Max Temp,Cloudiness,Humidity,Wind Speed,Hotel Name
0,Gao,ML,1569413201,16.28,-0.04,102.94,0,14,6.82,Hôtel Atlantide
1,Saint George,GR,1569413262,39.45,22.34,78.80,20,47,6.93,NaN
2,Kununurra,AU,1569413265,-15.77,128.74,75.20,0,38,5.48,Hotel Kununurra
3,Hermanus,ZA,1569413327,-34.42,19.24,84.00,0,32,3.00,Windsor Hotel
4,Taoudenni,ML,1569413333,22.68,-3.98,107.07,0,7,2.04,Bir Bekhera
5,Bilma,NE,1569413340,18.69,12.92,99.70,0,10,6.64,Bilma
6,Marrakesh,MA,1569413341,31.63,-7.99,89.60,20,29,5.82,Riad Adore by Pure Riads
7,Zaria,NG,1569413352,11.11,7.72,89.07,97,47,3.91,Luziana Hotel
8,Bunia,CD,1569413401,1.56,30.25,86.20,46,45,1.77,Hotel Karibuni
9,Malanje,AO,1569413463,-9.54,16.35,83.32,96,47,5.66,Hotel Portugalia


In [161]:
narrowed_city_df = hotel_df.dropna()
narrowed_city_df

,City,Country,Date,Latitude,Longitude,Max Temp,Cloudiness,Humidity,Wind Speed,Hotel Name
0,Gao,ML,1569413201,16.28,-0.04,102.94,0,14,6.82,Hôtel Atlantide
2,Kununurra,AU,1569413265,-15.77,128.74,75.20,0,38,5.48,Hotel Kununurra
3,Hermanus,ZA,1569413327,-34.42,19.24,84.00,0,32,3.00,Windsor Hotel
4,Taoudenni,ML,1569413333,22.68,-3.98,107.07,0,7,2.04,Bir Bekhera
5,Bilma,NE,1569413340,18.69,12.92,99.70,0,10,6.64,Bilma
6,Marrakesh,MA,1569413341,31.63,-7.99,89.60,20,29,5.82,Riad Adore by Pure Riads
7,Zaria,NG,1569413352,11.11,7.72,89.07,97,47,3.91,Luziana Hotel
8,Bunia,CD,1569413401,1.56,30.25,86.20,46,45,1.77,Hotel Karibuni
9,Malanje,AO,1569413463,-9.54,16.35,83.32,96,47,5.66,Hotel Portugalia
10,Conceicao do Araguaia,BR,1569413532,-8.26,-49.26,92.67,6,36,5.19,JAMBOS HOTEL


In [162]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = narrowed_city_df[["Latitude", "Longitude"]]

In [163]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,info_box_content = hotel_info)
# Add the layer to the map
fig.add_layer(markers)
fig

# Display Map

Figure(layout=FigureLayout(height='420px'))